In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np

In [ ]:
#loading data
data = pd.read_pickle("./DataSignal.pkl")


In [3]:
#loading the model based on Fawaz, Hassan Ismail, et al. "Transfer learning for time series classification." 2018 IEEE international conference on big data (Big Data) . IEEE, 2018.
num_classes =2
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

model = make_model((500, 1))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500, 1)]          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 500, 64)           256       
_________________________________________________________________
batch_normalization_3 (Batch (None, 500, 64)           256       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 500, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 64)           12352     
_________________________________________________________________
batch_normalization_4 (Batch (None, 500, 64)           256       
_________________________________________________________________
re_lu_4 (ReLU)               (None, 500, 64)           0   

In [55]:
#loading model weights
model.load_weights('/home/dell/Documents/master_thesis/best_model.h5')


In [56]:
#freezing feature extraction part of the network
for layer in model.layers[:-1]:
    layer.trainable=False
for layer in model.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd58c441208> False
<tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7fd5b82f6438> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fd58c4416a0> False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fd58c4419e8> False
<tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7fd5942bef98> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fd58c460400> False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fd58c4606d8> False
<tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7fd58c467668> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fd58c403198> False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fd58c4034e0> False
<tensorflow.python.keras.layers.pooling.GlobalAveragePooling1D object at 0x7fd58c408470>

In [65]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution1D, MaxPooling1D
model2=Sequential()
model2.add(model)
model2.add(Dense(1,activation='sigmoid'))
model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_3 (Functional)    (None, 2)                 25858     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 3         
Total params: 25,861
Trainable params: 133
Non-trainable params: 25,728
_________________________________________________________________


In [48]:
#Data preprocessing
data_F40 = data.loc[((data['Status'] == "OK")|(data['Status'] == "NOK"))&(data['Model'] == "F40")]

X_test = data_F40['Acc'].to_numpy()

X_test = np.array(X_test.tolist())

#X_test = np.reshape(x_test_temp, (x_test_temp.shape[0], x_test_temp.shape[1], 1))
y_test = LabelEncoder().fit_transform(data_files_all['Status'].to_numpy())

print(X_test.shape)
print(y_test.shape)

(4784, 3411)
(4784,)


In [50]:
from sklearn.model_selection import train_test_split
xtrain,xval,ytrain,yval=train_test_split(X_test,y_test,test_size=0.2,random_state=42)

xtrain=tf.convert_to_tensor(xtrain)
ytrain=tf.convert_to_tensor(ytrain)
xval=tf.convert_to_tensor(xval)
yval=tf.convert_to_tensor(yval)

In [68]:
#Training the model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(xtrain,ytrain, epochs=10)

Epoch 1/10


120/120 [==============================] - 7s 62ms/step - loss: nan - accuracy: 0.0510
Epoch 2/10
120/120 [==============================] - 8s 63ms/step - loss: nan - accuracy: 0.0510
Epoch 3/10
120/120 [==============================] - 8s 63ms/step - loss: nan - accuracy: 0.0510
Epoch 4/10
120/120 [==============================] - 7s 61ms/step - loss: nan - accuracy: 0.0510
Epoch 5/10
120/120 [==============================] - 7s 60ms/step - loss: nan - accuracy: 0.0510
Epoch 6/10
120/120 [==============================] - 7s 59ms/step - loss: nan - accuracy: 0.0510
Epoch 7/10
120/120 [==============================] - 8s 63ms/step - loss: nan - accuracy: 0.0510
Epoch 8/10
120/120 [==============================] - 8s 67ms/step - loss: nan - accuracy: 0.0510
Epoch 9/10
120/120 [==============================] - 8s 64ms/step - loss: nan - accuracy: 0.0510
Epoch 10/10
120/120 [==============================] - 8s 64ms/step - loss: nan - accuracy: 0.0510


In [69]:
y_pred = model2.predict(xval)



model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
loss, accuracy= model2.evaluate(xval,yval)

print('\nLoss: %.2f, Accuracy: %.2f%%' %(loss,accuracy*100) )

30/30 [==============================] - 2s 61ms/step - loss: nan - accuracy: 0.0449

Loss: nan, Accuracy: 4.49%


In [81]:
input_shape=(500,1)
X_train_resized = []
for x in xtrain:
    X_train_resized.append(np.resize(x, input_shape))

X_train_resized = np.array(X_train_resized)

print(X_train_resized.shape)
# resize test set
X_test_resized = []
for x in xval:
    X_test_resized.append(np.resize(x, input_shape))
X_test_resized = np.array(X_test_resized)

(3827, 500, 1)


In [76]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train_resized,ytrain, epochs=10)

Epoch 1/10
120/120 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.0510
Epoch 2/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 3/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 4/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 5/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 6/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 7/10
120/120 [==============================] - 1s 10ms/step - loss: nan - accuracy: 0.0510
Epoch 8/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510A: 0s - loss: na
Epoch 9/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510
Epoch 10/10
120/120 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.0510


In [82]:
y_pred = model2.predict(X_test_resized)



model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
loss, accuracy= model2.evaluate(X_test_resized,yval)

print('\nLoss: %.2f, Accuracy: %.2f%%' %(loss,accuracy*100) )

30/30 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0449

Loss: nan, Accuracy: 4.49%
